In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from numpy import random
from sklearn.datasets import load_digits
random.seed(1000)

In [4]:
def evalf(A,x,y,n_feat, n, lamb):
  f = np.matmul(A,x) - y
  return 0.5*(np.linalg.norm(f))**2  + 0.5*lamb*(np.matmul(x.T , x))


def evalg(A,x,y,n_feat, n, lamb):
  d = np.matmul(A,x) - y
  g = np.matmul(A.T , d) + lamb*x
  return g

def evalh(A,x,y,n_feat, n, lamb):
  h = np.matmul(A.T , A) + lamb*np.identity(n_feat)
  return h


In [5]:
def compute_steplength_backtracking_scaled_direction(A,x,y, n_feat, n ,lamb  ,gradf, direction, alpha_start, rho, gamma): #add appropriate arguments to the function


  alpha = alpha_start
  p = -gradf

  while (evalf(A, x + alpha*np.matmul(direction,p), y, n_feat, n,lamb) > (evalf(A,x,y,n_feat,n,lamb) + gamma * alpha * np.matmul(gradf.T, np.matmul(direction, p))) ):
    alpha = alpha*rho

  return alpha


#line search type
CONSTANT_STEP_LENGTH = 3
BACKTRACKING_LINE_SEARCH = 2
EXACT_LINE_SEARCH = 1



def find_minimizer_Newtonmethod(A, start_x, y, n_feat,  n,lamb, tol, line_search_type, *args):


  x = start_x
  g_x = evalg(A,x,y,n_feat,n,lamb)


  if line_search_type == BACKTRACKING_LINE_SEARCH:
    if args is None:
      err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive any args. Please check!'
      raise ValueError(err_msg)
    elif len(args)<3 :
      err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive three args. Please check!'
      raise ValueError(err_msg)
    else:
      alpha_start = float(args[0])
      rho = float(args[1])
      gamma = float(args[2])
  k = 0
  x_k = []



  while (np.linalg.norm(g_x) > tol):

    d = np.linalg.inv(evalh(A,x,y,n_feat,n,lamb))

    if line_search_type == BACKTRACKING_LINE_SEARCH:
      step_length = compute_steplength_backtracking_scaled_direction(A,x,y,n_feat,n,lamb,g_x, d , alpha_start,rho, gamma)

    elif line_search_type == CONSTANT_STEP_LENGTH:
      step_length = 1.0

    else:
      raise ValueError('Line search type unknown. Please check!')

    x = np.subtract(x, step_length * np.matmul(d,g_x))
    k += 1
    x_k.append(x)
    g_x = evalg(A,x,y,n_feat,n,lamb)

  return x, k, x_k



def find_minimizer_BFGS(A, start_x, y, n_feat, n,lamb, tol,line_search_type,*args):



  x = start_x
  k = 0
  g_new = evalg(A,x,y,n_feat,n,lamb)
  B = np.identity(n_feat)
  x_k = []

  if line_search_type == BACKTRACKING_LINE_SEARCH:
    if args is None:
      err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive any args. Please check!'
      raise ValueError(err_msg)
    elif len(args)<3 :
      err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive three args. Please check!'
      raise ValueError(err_msg)
    else:
      alpha_start = float(args[0])
      rho = float(args[1])
      gamma = float(args[2])

  while (np.linalg.norm(g_new) > tol):

    d = B

    if line_search_type == BACKTRACKING_LINE_SEARCH:
      step_length = compute_steplength_backtracking_scaled_direction(A,x,y,n_feat,n,lamb ,g_new, d , alpha_start,rho, gamma)

    elif line_search_type == CONSTANT_STEP_LENGTH:
      step_length = 1.0

    else:
      raise ValueError('Line search type unknown. Please check!')

    g_old = evalg(A,x,y,n_feat,n,lamb)
    p = np.matmul(-1*B, g_old)
    x = x + step_length * p
    s = step_length * p
    g_new = evalg(A,x,y,n_feat,n,lamb)
    y_k = g_new - g_old

    mu = 1 / (np.matmul(y_k.T,s))
    term1 = np.identity(n_feat) - mu * np.matmul(s,y_k.T)
    term2 = np.identity(n_feat) - mu * np.matmul(y_k, s.T)

    B = np.matmul(term1, np.matmul(B ,term2))  +  mu * np.matmul(s,s.T)

    k = k + 1
    x_k.append(x)


  return x, k, x_k

In [6]:
digits = load_digits()

print(digits.data.shape)
print(digits.target.shape)

N = digits.data.shape[0]
d = digits.data.shape[1]
A = digits.data
y = 1.0 * np.ones([A.shape[0],1])

for i in range(digits.target.shape[0]):
  y[i] = digits.target[i]


(1797, 64)
(1797,)


***1. Use Newton’s method to solve the problem (1), which is direct OLSLR. Use the starting point x = 0. Report
the difficulties, you encounter. Check if these difficulties remain if you use Newton’s method to solve the
problem (2), the regularized OLSLR with λ = 0.001 and the same starting point. Explain the reasons for your
observation. Report the values of $x^*_f$
and $x^*_{f_{\lambda}}$.***

In [7]:
my_x = np.zeros((A.shape[1],1))
lamb = [0, 0.001]
my_tol = 10e-5
alpha = 0.9
rho = 0.5
gamma = 0.5

try:
  x_0 , k_0 , x_k0 = find_minimizer_Newtonmethod(A,my_x,y,d,N,lamb[0],my_tol,BACKTRACKING_LINE_SEARCH, alpha, rho, gamma)
except Exception as e:
  print('Error : ',e)

x_1 , k_1 , x_k1 = find_minimizer_Newtonmethod(A,my_x,y,d,N,lamb[1],my_tol,BACKTRACKING_LINE_SEARCH, alpha, rho, gamma)

print('Results :-')
print('---')

print('With lamda=0 , ERROR : Singular matrix \n ')
print('With lambda=0.001 , x_opt = ',x_1.T , '\n\n and the number of iterations taken is,' , k_1)

Error :  Singular matrix
Results :-
---
With lamda=0 , ERROR : Singular matrix 
 
With lambda=0.001 , x_opt =  [[ 0.00000000e+00  9.69081198e-02 -4.32184332e-03 -7.75905151e-03
   7.49591748e-02  1.13946521e-02 -2.71290491e-02 -7.34533997e-03
   9.98260900e-01 -2.88088759e-02  1.18688363e-01  6.60923364e-02
  -5.57076519e-02 -6.97055394e-02  9.65840774e-02  2.55197388e-01
  -7.29830858e-01  2.42711927e-02  7.73240740e-02 -2.33009319e-02
  -5.64076767e-02  5.72412687e-02 -4.88671008e-02 -2.62564814e-01
  -9.06022045e-01 -1.49769823e-01  5.64023115e-02  8.96665491e-02
   8.39315716e-02  9.85409858e-02  1.69265074e-03 -2.96634092e+00
   0.00000000e+00 -1.54361384e-01 -9.32408982e-03  1.39498013e-01
  -3.69237696e-02  5.46111772e-02 -9.20417645e-03  0.00000000e+00
   1.03331195e-01  1.23983507e-01 -1.37634792e-02  5.40023150e-03
   1.31185759e-01  5.49578519e-02  2.24935666e-02  7.48053084e-03
   6.17483072e-01  2.44098450e-02  1.42358774e-03 -6.21115205e-02
  -2.07024590e-01 -3.38510968e-

 ***2. Use the BFGS method with starting point x = 0, to solve the problem (1) and describe if you observe any
difficulty. Check if solving the regularized problem (2) helps (use λ = 0.001 and starting point x = 0). Explain
your observations. Report the values of $x^*_f$
and $x^*_{f_{\lambda}}$.***


In [9]:
x_0 , k_0 , x_k0 = find_minimizer_BFGS(A,my_x,y,d,N,lamb[1],my_tol,BACKTRACKING_LINE_SEARCH, alpha, rho, gamma)
x_1 , k_1 , x_k1 = find_minimizer_BFGS(A,my_x,y,d,N,lamb[1],my_tol,BACKTRACKING_LINE_SEARCH, alpha, rho, gamma)

print('For the BFGS Methods Without regualrization')
print('\n')
print('Optimal x : ', x_0.T)
print('Number of itertion : ', k_0)

print('--------')

print('For the BFGS Methods With lambda = 0.001')
print('\n')
print('Optimal x : ', x_1.T)
print('Number of itertion : ', k_1)

For the BFGS Methods Without regualrization


Optimal x :  [[ 0.00000000e+00  9.69081206e-02 -4.32184354e-03 -7.75905153e-03
   7.49591748e-02  1.13946521e-02 -2.71290491e-02 -7.34533995e-03
   9.98260900e-01 -2.88088760e-02  1.18688363e-01  6.60923365e-02
  -5.57076520e-02 -6.97055394e-02  9.65840773e-02  2.55197389e-01
  -7.29830856e-01  2.42711928e-02  7.73240741e-02 -2.33009319e-02
  -5.64076767e-02  5.72412688e-02 -4.88671008e-02 -2.62564815e-01
  -9.06022046e-01 -1.49769823e-01  5.64023114e-02  8.96665492e-02
   8.39315716e-02  9.85409858e-02  1.69265083e-03 -2.96634092e+00
   0.00000000e+00 -1.54361384e-01 -9.32408980e-03  1.39498013e-01
  -3.69237696e-02  5.46111772e-02 -9.20417641e-03  0.00000000e+00
   1.03331196e-01  1.23983507e-01 -1.37634792e-02  5.40023149e-03
   1.31185759e-01  5.49578520e-02  2.24935666e-02  7.48053130e-03
   6.17483070e-01  2.44098451e-02  1.42358773e-03 -6.21115205e-02
  -2.07024590e-01 -3.38510968e-02  1.05486790e-01 -1.40336534e-01
  -9.82165963e-01